In [1]:
!pip install -q google-generativeai transformers tensorflow keras PyMuPDF huggingface-hub evaluate accelerate gradio bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 85.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 33.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 100.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.

In [14]:
import os
import gc
import csv
import fitz  # PyMuPDF
import torch
import pandas as pd
import gradio as gr
import google.generativeai as genai
from huggingface_hub import HfFolder
from kaggle_secrets import UserSecretsClient
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Environment setup
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
gc.collect()
torch.cuda.empty_cache()

In [15]:
secrets = UserSecretsClient()
genai.configure(api_key=secrets.get_secret("API_KEY"))
HfFolder.save_token(secrets.get_secret("HF_TOKEN"))

In [16]:
import os
import pandas as pd
import csv
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import tensorflow as tf
def load_models():
    model1 = None
    if os.path.exists("/kaggle/input/diabetes/tensorflow2/default/1/mlp_model_weights.weights.h5"):
        with tf.device('/CPU:0'):   # ← Force-load on CPU
            model1 = Sequential([
                Dense(256, activation='relu', input_shape=(5,)),  # Adjusted input shape to match 5 features
                Dropout(0.3),
                Dense(128, activation='relu'),
                Dropout(0.3),
                Dense(64, activation='relu'),
                Dropout(0.3),
                Dense(1, activation='sigmoid')
            ])
            model1.load_weights('/kaggle/input/diabetes/tensorflow2/default/1/mlp_model_weights.weights.h5')
    return model1

model1 = load_models()

# -------------------------
#  PDF PARSING FUNCTIONS
# -------------------------

import fitz  # PyMuPDF
import google.generativeai as genai

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file while preserving horizontal structure."""
    doc = fitz.open(pdf_path)
    extracted_text = []
    for page in doc:
        blocks = page.get_text("blocks")  # Extract text in blocks
        sorted_blocks = sorted(blocks, key=lambda b: (b[1], b[0]))  # Sort by vertical then horizontal
        page_text = "\n".join(block[4] for block in sorted_blocks)
        extracted_text.append(page_text)
    return "\n".join(extracted_text)

def structure_text_with_gemini(text):
    """
    Send extracted text to Gemini for tabular structuring and CSV formatting,
    including detailed status analysis.
    """
    model_1 = genai.GenerativeModel("gemini-2.0-flash")

    prompt = (
        "Extract structured tabular data from the following medical report and format it as CSV with proper headers.\n"
        "Ensure the column headers exactly match the following list:\n"
        "age, bmi, HbA1c_level, blood_glucose_level, diabetes\n\n"
        "Return all values numerically in CSV format with one row.\n\n"
        "After the CSV, also detect whether the patient shows signs of **anemia**, based on any of the following:\n"
        "- Low Hemoglobin (e.g., <13.0 for men, <12.0 for women)\n"
        "- Low Hematocrit\n"
        "- Descriptions like 'anemic', 'iron deficiency', etc.\n"
        "Append a final line like this: anemia_flag: 1 (if anemia detected), otherwise anemia_flag: 0.\n\n"
        f"{text}"
    )
    response = model_1.generate_content(prompt)
    return response.text.strip() if response.text else ""

def save_csv(csv_text, output_csv):
    """Save the structured CSV text into a file with consistent columns."""
    rows = [line.split(",") for line in csv_text.split("\n") if line.strip()]
    max_cols = max(len(row) for row in rows) if rows else 0
    normalized_rows = [row + [""] * (max_cols - len(row)) for row in rows]
    with open(output_csv, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        writer.writerows(normalized_rows)

def get_model():
    return secrets.get_secret("Models")

def pdf_to_csv(pdf_path, output_csv):
    """Complete pipeline: Extract, structure, and save as CSV."""
    text = extract_text_from_pdf(pdf_path)
    csv_text = structure_text_with_gemini(text)
    save_csv(csv_text, output_csv)
    return output_csv

# -------------------------
#  MAPPING FUNCTIONS
# -------------------------

def map_data_for_model1(csv_file):
    """
    Read the CSV produced from the PDF and map its data to the input features for model 1.
    If some values are missing, use safe default values.
    """
    defaults = {
        "age": 43,
        "bmi": 27.32,
        "HbA1c_level": 5.8,
        "blood_glucose_level": 140.0,
        "diabetes": 0
    }
    try:
        df = pd.read_csv(csv_file)
        model1_input = {
            key: df[key].iloc[0] if key in df.columns and not pd.isna(df[key].iloc[0]) else default
            for key, default in defaults.items()
        }
    except Exception as e:
        print("Error reading CSV: " + str(e))
        model1_input = defaults
    return model1_input


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
bnb_config = BitsAndBytesConfig(
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)
)

model_id = "epfl-llm/meditron-7b"

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [25]:
def generate_meditron_prompt(structured_csv, diabetes_result):
    return f"""
You are a clinical report generator using blood biomarkers. Your task is to generate a structured, medically accurate patient report based ONLY on the following lab data and diabetes risk assessment.

Structured Lab Data:
{structured_csv}

Diabetes prediction (from independent model): {diabetes_result}

Write a detailed report including:
1. Potential health vulnerabilities (CBC, Lipid Profile, etc.)
2. Suggested lifestyle changes and dietary guidance.
3. Natural/home remedies (if relevant).
4. Recommended medical specialists to consult.

- Format in professional bullet points.
- Limit to **approximately 500 tokens**.
- Be concise, medical, and actionable.
- Don't use markdown, stick to simple text.
"""


import tensorflow as tf

def predict_diabetes_cpu(model, input_df):
    with tf.device('/CPU:0'):
        return model.predict(input_df)

def generate_report(pdf_file):
    pdf_path = pdf_file.name
    
    extracted_text = extract_text_from_pdf(pdf_path)
    structured_text = structure_text_with_gemini(extracted_text)
    
    temp_csv_path = "structured_report.csv"
    save_csv(structured_text, temp_csv_path)
    model1_input = map_data_for_model1(temp_csv_path)

    # Strict feature order
    FEATURE_ORDER = ["age", "bmi", "HbA1c_level", "blood_glucose_level", "diabetes"]
    input_df = pd.DataFrame([model1_input])[FEATURE_ORDER]
    
    # Predict diabetes safely
    with tf.device('/CPU:0'):
        diabetes_prob = model1.predict(input_df)[0][0]

    diabetes_result = "Positive (high risk)" if diabetes_prob >= 0.5 else "Negative (low risk)"
    
    prompt = generate_meditron_prompt(structured_text, diabetes_result)

    model_1 = genai.GenerativeModel("gemini-2.0-flash")
    response = model_1.generate_content(prompt)
    
    # Clean up Gemini output
    generated_text = response.text.strip()
    if "<END>" in generated_text:
        generated_text = generated_text.split("<END>")[0].strip()
    
    final_report = generated_text + f"\n\n🩺 **Diabetes Prediction:** {diabetes_result}"

    with open("generated_report.txt", "w") as f:
        f.write(final_report)

    return final_report, "generated_report.txt"


In [26]:
interface = gr.Interface(
    fn=generate_report,
    inputs=gr.File(label="Upload Medical Report PDF", file_types=[".pdf"]),
    outputs=[
        gr.Textbox(label="Generated Medical Report"),
        gr.File(label="Download Report")
    ],
    title="🧠 Meditron-7B Medical Report Generator",
    description="Upload a CBC report PDF. The app extracts key features, analyzes them with Meditron-7B, and generates a ~500-token structured medical report."
)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://b9db247731905b3e8c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
